In [ ]:
import re
import sys
from collections import defaultdict

# ==============================================================================
# 1. ANALISADOR LÉXICO (SCANNER)
# ==============================================================================
class Lexer:
    def __init__(self, code):
        self.code = code
        self.token_specs = [
            ('MAIN',   r'\bmain\b'),
            ('INT',    r'\bint\b'),
            ('FLOAT',  r'\bfloat\b'),
            ('DOUBLE', r'\bdouble\b'),
            ('CHAR',   r'\bchar\b'),
            ('BOOL',   r'\bbool\b'),
            ('STRING', r'\bstring\b'),

            ('IF',     r'\bif\b'),
            ('ELSE',   r'\belse\b'),
            ('WHILE',  r'\bwhile\b'),
            ('DO',     r'\bdo\b'),
            ('FOR',    r'\bfor\b'),
            ('SWITCH', r'\bswitch\b'),
            ('CASE',   r'\bcase\b'),
            ('DEFAULT',r'\bdefault\b'),
            ('BREAK',  r'\bbreak\b'),
            ('RETURN', r'\breturn\b'),

            ('COMMENT', r'//.*'),

            ('EQ',     r'=='),
            ('NEQ',    r'!='),
            ('LTE',    r'<=' ),
            ('GTE',    r'>=' ),
            ('AND',    r'&&'),
            ('OR',     r'\|\|'),

            ('LT',     r'<'),
            ('GT',     r'>'),
            ('NOT',    r'!'),

            ('PLUS',   r'\+'),
            ('MINUS',  r'-'),
            ('MULT',   r'\*'),
            ('DIV',    r'/'),
            ('MOD',    r'%'),
            ('ASSIGN', r'='),

            ('LPAREN', r'\('),
            ('RPAREN', r'\)'),
            ('LCURLY', r'\{'),
            ('RCURLY', r'\}'),
            ('SEMI',   r';'),
            ('COMMA',  r','),
            ('COLON',  r':'),

            # ==============================
            #   LITERAIS
            # ==============================

            ('CHAR_LITERAL',   r"'([^'\\]|\\.)'"),
            ('STRING_LITERAL', r'"([^"\\]|\\.)*"'),
            ('BOOL_LITERAL',   r'\b(true|false)\b'),

            ('NUMBER', r'\d+(\.\d+)?'),
            ('ID',     r'[a-zA-Z_][a-zA-Z_0-9]*'),

            ('WS',     r'\s+'),
            ('MISMATCH', r'.'),
        ]

    def tokenize(self):

        tokens = []

        # Monta uma única expressão regular contendo TODAS as regras de tokens.
        token_regex = '|'.join(f'(?P<{pair[0]}>{pair[1]})' for pair in self.token_specs)

        line_num = 1

        # Percorre o código fonte procurando todos os matches das regex combinadas
        for match in re.finditer(token_regex, self.code):

            # Pega o nome do token reconhecido
            kind = match.lastgroup

            # Pega o texto exato encontrado
            value = match.group()

            if kind == 'WS':
                line_num += value.count('\n')
                continue


            # captura qualquer caractere não reconhecido por nenhuma regra.
            elif kind == 'MISMATCH':
                print(f"ERRO LÉXICO: Caractere inesperado '{value}' na linha {line_num}")
                sys.exit(1)

            elif kind == 'COMMENT':
                continue

            else:
                tokens.append((kind, value))

        tokens.append(('EOF', '$'))

        return tokens


# ==============================================================================
# 2. GRAMÁTICA LL(1)
# ==============================================================================

grammar = {
    'PROGRAM': [['MAIN_FUNCTION', 'EOF']],

    'MAIN_FUNCTION': [['MAIN', 'LPAREN', 'RPAREN', 'LCURLY', 'STATEMENT_LIST', 'RCURLY']],

    'STATEMENT_LIST': [
        ['STATEMENT', 'STATEMENT_LIST'],
        ['EPSILON']
    ],

    'STATEMENT': [
        ['DECLARATION'],
        ['IF_STATEMENT'],
        ['WHILE_STATEMENT'],
        ['DO_WHILE_STATEMENT'],
        ['FOR_STATEMENT'],
        ['SWITCH_STATEMENT'],
        ['RETURN_STATEMENT'],
        ['STATEMENT_ID_START']
    ],

    'STATEMENT_ID_START': [
        ['ID', 'STATEMENT_ID_TAIL']
    ],

    'STATEMENT_ID_TAIL': [
        ['ASSIGN', 'EXPRESSION', 'SEMI'],
        ['TERM_PRIME', 'EXP_PRIME', 'SEMI']
    ],

    'DECLARATION': [['TYPE', 'ID', 'DECLARATION_TAIL']],

    'DECLARATION_TAIL': [
        ['ASSIGN', 'EXPRESSION', 'SEMI'],
        ['SEMI']
    ],

    'TYPE': [['INT'], ['FLOAT'], ['DOUBLE'], ['CHAR'], ['BOOL'], ['STRING']],

    'IF_STATEMENT': [['IF', 'LPAREN', 'CONDITION', 'RPAREN', 'LCURLY', 'STATEMENT_LIST', 'RCURLY', 'ELSE_CLAUSE']],

    'ELSE_CLAUSE': [
        ['ELSE', 'LCURLY', 'STATEMENT_LIST', 'RCURLY'],
        ['EPSILON']
    ],

    'WHILE_STATEMENT': [['WHILE', 'LPAREN', 'CONDITION', 'RPAREN', 'LCURLY', 'STATEMENT_LIST', 'RCURLY']],

    'DO_WHILE_STATEMENT': [
        ['DO', 'LCURLY', 'STATEMENT_LIST', 'RCURLY', 'WHILE', 'LPAREN', 'CONDITION', 'RPAREN', 'SEMI']
    ],

    'FOR_STATEMENT': [['FOR', 'LPAREN', 'FOR_INIT', 'SEMI', 'CONDITION', 'SEMI', 'FOR_UPDATE', 'RPAREN', 'LCURLY', 'STATEMENT_LIST', 'RCURLY']],

    'FOR_INIT': [['DECLARATION'], ['ASSIGNMENT'], ['EPSILON']],
    'ASSIGNMENT': [['ID', 'ASSIGN', 'EXPRESSION']],
    'FOR_UPDATE': [['ID', 'ASSIGN', 'EXPRESSION'], ['EPSILON']],



    'SWITCH_STATEMENT': [
        ['SWITCH', 'LPAREN', 'EXPRESSION', 'RPAREN', 'LCURLY', 'CASE_LIST', 'RCURLY']
    ],

    'CASE_LIST': [
        ['CASE', 'LITERAL', 'COLON', 'STATEMENT_LIST', 'BREAK', 'SEMI', 'CASE_LIST'],
        ['DEFAULT', 'COLON', 'STATEMENT_LIST', 'BREAK', 'SEMI'],
        ['EPSILON']
    ],

    'LITERAL': [
        ['NUMBER'],
        ['CHAR_LITERAL'],
        ['STRING_LITERAL'],
        ['BOOL_LITERAL']
    ],

    'RETURN_STATEMENT': [['RETURN', 'RETURN_TAIL']],

    'RETURN_TAIL': [['EXPRESSION', 'SEMI'], ['SEMI']],

    'CONDITION': [['LOGICAL_OR_EXPR']],

    'LOGICAL_OR_EXPR': [['LOGICAL_AND_EXPR', 'LOGICAL_OR_PRIME']],
    'LOGICAL_OR_PRIME': [['OR', 'LOGICAL_AND_EXPR', 'LOGICAL_OR_PRIME'], ['EPSILON']],

    'LOGICAL_AND_EXPR': [['RELATIONAL_EXPR', 'LOGICAL_AND_PRIME']],
    'LOGICAL_AND_PRIME': [['AND', 'RELATIONAL_EXPR', 'LOGICAL_AND_PRIME'], ['EPSILON']],

    'RELATIONAL_EXPR': [['EXPRESSION', 'RELATIONAL_TAIL'], ['NOT', 'RELATIONAL_EXPR']],

    'RELATIONAL_TAIL': [
        ['EQ', 'EXPRESSION'], ['NEQ', 'EXPRESSION'],
        ['LT', 'EXPRESSION'], ['GT', 'EXPRESSION'],
        ['LTE', 'EXPRESSION'], ['GTE', 'EXPRESSION'],
        ['EPSILON']
    ],

    'EXPRESSION': [['TERM', 'EXP_PRIME']],

    'EXP_PRIME': [
        ['PLUS', 'TERM', 'EXP_PRIME'],
        ['MINUS', 'TERM', 'EXP_PRIME'],
        ['EPSILON']
    ],

    'TERM': [['FACTOR', 'TERM_PRIME']],

    'TERM_PRIME': [
        ['MULT', 'FACTOR', 'TERM_PRIME'],
        ['DIV', 'FACTOR', 'TERM_PRIME'],
        ['MOD', 'FACTOR', 'TERM_PRIME'],
        ['EPSILON']
    ],

    'FACTOR': [
        ['ID'],
        ['NUMBER'],
        ['CHAR_LITERAL'],
        ['STRING_LITERAL'],
        ['BOOL_LITERAL'],
        ['LPAREN', 'EXPRESSION', 'RPAREN'],
        ['MINUS', 'FACTOR']
    ]
}

terminals = {
    'MAIN', 'INT', 'FLOAT', 'DOUBLE','CHAR', 'BOOL', 'STRING',
    'CHAR_LITERAL', 'STRING_LITERAL', 'BOOL_LITERAL',
    'IF', 'ELSE', 'WHILE', 'DO', 'FOR',
    'SWITCH', 'CASE', 'DEFAULT', 'BREAK',
    'RETURN', 'COMMENT',
    'ID', 'NUMBER', 'PLUS', 'MINUS', 'MULT', 'DIV', 'MOD', 'ASSIGN',
    'EQ', 'NEQ', 'LT', 'GT', 'LTE', 'GTE', 'AND', 'OR', 'NOT',
    'LPAREN', 'RPAREN', 'LCURLY', 'RCURLY', 'SEMI', 'COMMA', 'COLON', 'EOF'
}
non_terminals = set(grammar.keys())

# ==============================================================================
# 3. FIRST, FOLLOW, TABELA PREDITIVA
# ==============================================================================

first = defaultdict(set)
follow = defaultdict(set)
parsing_table = {}

def compute_first():
    global first
    for t in terminals:
        first[t].add(t)
    first['EPSILON'].add('EPSILON')

    changed = True
    while changed:
        changed = False
        for head, productions in grammar.items():
            for prod in productions:
                can_be_empty = True
                for symbol in prod:
                    initial_len = len(first[head])
                    s_first = first[symbol].copy()
                    if 'EPSILON' in s_first:
                        s_first.remove('EPSILON')
                    else:
                        can_be_empty = False
                    first[head].update(s_first)
                    if len(first[head]) > initial_len:
                        changed = True
                    if not can_be_empty:
                        break
                if can_be_empty:
                    if 'EPSILON' not in first[head]:
                        first[head].add('EPSILON')
                        changed = True

def compute_follow():
    global follow
    follow['PROGRAM'].add('$')
    changed = True
    while changed:
        changed = False
        for head, productions in grammar.items():
            for prod in productions:
                trailer = follow[head].copy()
                for symbol in reversed(prod):
                    if symbol in non_terminals:
                        initial_len = len(follow[symbol])
                        follow[symbol].update(trailer)
                        if len(follow[symbol]) > initial_len:
                            changed = True
                        if 'EPSILON' in first[symbol]:
                            trailer.update(first[symbol])
                            trailer.remove('EPSILON')
                        else:
                            trailer = first[symbol].copy()
                    else:
                        trailer = first[symbol].copy()

def build_parsing_table():
    global parsing_table
    for head, productions in grammar.items():
        for prod in productions:
            prod_first = set()
            can_be_empty = True
            for symbol in prod:
                s_first = first[symbol].copy()
                if 'EPSILON' in s_first:
                    s_first.remove('EPSILON')
                else:
                    can_be_empty = False
                prod_first.update(s_first)
                if not can_be_empty:
                    break

            for terminal in prod_first:
                parsing_table[(head, terminal)] = prod

            if can_be_empty or (len(prod) == 1 and prod[0] == 'EPSILON'):
                for terminal in follow[head]:
                    parsing_table[(head, terminal)] = prod

# ==============================================================================
# 4. PARSER MANUAL COM PILHA
# ==============================================================================

def parse(tokens):

    stack = ['PROGRAM']

    # Cursor aponta para o token atual da entrada
    cursor = 0

    print(f"\n{'PILHA':<70} | {'ENTRADA':<20} | AÇÃO")
    print("-" * 110)

    # Loop principal do parser — roda enquanto houver símbolos na pilha
    while len(stack) > 0:
        # "top" é o topo da pilha
        top = stack[-1]

        # Proteção: cursor nunca deve ultrapassar os tokens disponíveis
        if cursor >= len(tokens):
            print("ERRO CRÍTICO: Tentativa de ler além do fim.")
            return False

        # Token atual: tipo e valor (ex.: 'ID', 'x')
        token_type = tokens[cursor][0]
        token_val = tokens[cursor][1]

        # Para imprimir o estado atual da pilha
        stack_str = " ".join(stack)


        # Se o topo da pilha é um terminal e é igual ao token atual:
        # simplesmente consome o token.
        if top == token_type:

            # Caso especial: EOF encontrado → fim da análise, sucesso total.
            if top == 'EOF':
                print(f"{stack_str:<70} | {token_val:<20} | SUCESSO: ACEITO")
                return True

            # Consome token e desempilha o terminal
            print(f"{stack_str:<70} | {token_val:<20} | Ler {top}")
            stack.pop()
            cursor += 1


        # Se o topo da pilha é 'EPSILON', apenas removemos.
        elif top == 'EPSILON':
            print(f"{stack_str:<70} | {token_val:<20} | Epsilon")
            stack.pop()

        # ----------------------------------------------------------------------
        # 3) CASO 3: ERRO — TERMINAL ESPERADO, MAS NÃO BATEU
        # ----------------------------------------------------------------------
        # Se o topo é terminal mas não casa com o token atual → erro sintático.
        elif top in terminals:
            print(f"\nERRO: Esperado '{top}', encontrado '{token_val}'")
            return False

        # ----------------------------------------------------------------------
        # 4) CASO 4: USAR TABELA PREDITIVA LL(1)
        # ----------------------------------------------------------------------
        # Se existe uma regra para essa combinação (não-terminal, token), aplica.
        elif (top, token_type) in parsing_table:
            prod = parsing_table[(top, token_type)]

            # Mostra qual produção está sendo usada
            print(f"{stack_str:<70} | {token_val:<20} | {top} -> {' '.join(prod)}")

            # Remove o não-terminal da pilha
            stack.pop()

            # Empilha a produção ao contrário (pois a pilha é LIFO)
            for symbol in reversed(prod):
                stack.append(symbol)

        # ----------------------------------------------------------------------
        # 5) CASO 5: ERRO — NÃO EXISTE REGRA NA TABELA
        # ----------------------------------------------------------------------
        else:
            print(f"\nERRO: Sem regra para {top} com {token_type}")
            return False

    # Se a pilha esvaziou e não deu erro, aceitou
    return True

In [ ]:
# ==============================================================================
# 1. ✔ SUCESSO – CÓDIGO ACEITO
# ==============================================================================
if __name__ == "__main__":

    codigo_teste = """
   main() {

    // =======================
    // DECLARAÇÕES
    // =======================
    int a = 10;
    int b = 20;
    bool flag = true;
    char letra = 'x';
    string msg = "teste";
    float x = 3.5;

    // =======================
    // IF / ELSE
    // =======================
    if(a < b) {
        a = a + 1;
    } else {
        b = b + 1;
    }

    // =======================
    // WHILE
    // =======================
    while(a < 15) {
        a = a + 1;
    }

    // =======================
    // DO / WHILE
    // =======================
    do {
        b = b - 1;
    } while(b > 10);

    // =======================
    // FOR
    // =======================
    for(i = 0; i < 5; i = i + 1) {
        x = x * 2;
    }

    // =======================
    // SWITCH / CASE / DEFAULT
    // =======================
    switch(a) {
        case 10:
            msg = "valor 10";
            break;

        case 15:
            msg = "valor 15";
            break;

        default:
            msg = "outro valor";
            break;
    }

    // =======================
    // RETURN
    // =======================
    return a;
}

    """


    print("=== 1. LEXER ===")
    lexer = Lexer(codigo_teste)
    tokens = lexer.tokenize()
    print("Tokens OK.")

    print("\n=== 2. TABLE ===")
    compute_first()
    compute_follow()
    build_parsing_table()
    print("Tabela OK.")

    print("\n=== 3. PARSER ===")
    parse(tokens)

=== 1. LEXER ===
Tokens OK.

=== 2. TABLE ===
Tabela OK.

=== 3. PARSER ===

PILHA                                                                  | ENTRADA              | AÇÃO
--------------------------------------------------------------------------------------------------------------
PROGRAM                                                                | main                 | PROGRAM -> MAIN_FUNCTION EOF
EOF MAIN_FUNCTION                                                      | main                 | MAIN_FUNCTION -> MAIN LPAREN RPAREN LCURLY STATEMENT_LIST RCURLY
EOF RCURLY STATEMENT_LIST LCURLY RPAREN LPAREN MAIN                    | main                 | Ler MAIN
EOF RCURLY STATEMENT_LIST LCURLY RPAREN LPAREN                         | (                    | Ler LPAREN
EOF RCURLY STATEMENT_LIST LCURLY RPAREN                                | )                    | Ler RPAREN
EOF RCURLY STATEMENT_LIST LCURLY                                       | {                    | Ler LCURLY

In [ ]:
# ==============================================================================
# 2. ❌ ERRO SINTÁTICO: Esperado SEMI(;), encontrado x
# ==============================================================================
if __name__ == "__main__":

    codigo_teste = """
    main() {
        int x = 10
        x = x + 1;
    }
    """

    print("=== 1. LEXER ===")
    lexer = Lexer(codigo_teste)
    tokens = lexer.tokenize()
    print("Tokens OK.")

    print("\n=== 2. TABLE ===")
    compute_first()
    compute_follow()
    build_parsing_table()
    print("Tabela OK.")

    print("\n=== 3. PARSER ===")
    parse(tokens)

=== 1. LEXER ===
Tokens OK.

=== 2. TABLE ===
Tabela OK.

=== 3. PARSER ===

PILHA                                                                  | ENTRADA              | AÇÃO
--------------------------------------------------------------------------------------------------------------
PROGRAM                                                                | main                 | PROGRAM -> MAIN_FUNCTION EOF
EOF MAIN_FUNCTION                                                      | main                 | MAIN_FUNCTION -> MAIN LPAREN RPAREN LCURLY STATEMENT_LIST RCURLY
EOF RCURLY STATEMENT_LIST LCURLY RPAREN LPAREN MAIN                    | main                 | Ler MAIN
EOF RCURLY STATEMENT_LIST LCURLY RPAREN LPAREN                         | (                    | Ler LPAREN
EOF RCURLY STATEMENT_LIST LCURLY RPAREN                                | )                    | Ler RPAREN
EOF RCURLY STATEMENT_LIST LCURLY                                       | {                    | Ler LCURLY

In [ ]:
# ==============================================================================
# 3. ❌ ERRO SINTÁTICO: Faltou } antes do EOF
# ==============================================================================
if __name__ == "__main__":

    codigo_teste = """
    main() {
        int x = 10;

        while (x < 5) {
            x = x + 1;
    """

    print("=== 1. LEXER ===")
    lexer = Lexer(codigo_teste)
    tokens = lexer.tokenize()
    print("Tokens OK.")

    print("\n=== 2. TABLE ===")
    compute_first()
    compute_follow()
    build_parsing_table()
    print("Tabela OK.")

    print("\n=== 3. PARSER ===")
    parse(tokens)

=== 1. LEXER ===
Tokens OK.

=== 2. TABLE ===
Tabela OK.

=== 3. PARSER ===

PILHA                                                                  | ENTRADA              | AÇÃO
--------------------------------------------------------------------------------------------------------------
PROGRAM                                                                | main                 | PROGRAM -> MAIN_FUNCTION EOF
EOF MAIN_FUNCTION                                                      | main                 | MAIN_FUNCTION -> MAIN LPAREN RPAREN LCURLY STATEMENT_LIST RCURLY
EOF RCURLY STATEMENT_LIST LCURLY RPAREN LPAREN MAIN                    | main                 | Ler MAIN
EOF RCURLY STATEMENT_LIST LCURLY RPAREN LPAREN                         | (                    | Ler LPAREN
EOF RCURLY STATEMENT_LIST LCURLY RPAREN                                | )                    | Ler RPAREN
EOF RCURLY STATEMENT_LIST LCURLY                                       | {                    | Ler LCURLY

In [ ]:
# ==============================================================================
# 4. ❌ ERRO LÉXICO: caractere inesperado $
# ==============================================================================
if __name__ == "__main__":

    codigo_teste = """
    main() {
        int x = 10;
        x = x $ 2;
    }
    """

    print("=== 1. LEXER ===")
    lexer = Lexer(codigo_teste)
    tokens = lexer.tokenize()
    print("Tokens OK.")

    print("\n=== 2. TABLE ===")
    compute_first()
    compute_follow()
    build_parsing_table()
    print("Tabela OK.")

    print("\n=== 3. PARSER ===")
    parse(tokens)

=== 1. LEXER ===
ERRO LÉXICO: Caractere inesperado '$' na linha 4


SystemExit: 1

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print("\n>>> CONJUNTO FIRST <<<")
for nao_terminal, conjunto in first.items():
    # Só imprime os Não-Terminais (regras) para não poluir
    if nao_terminal in grammar:
        print(f"FIRST({nao_terminal}) = {conjunto}")


>>> CONJUNTO FIRST <<<
FIRST(PROGRAM) = {'MAIN'}
FIRST(MAIN_FUNCTION) = {'MAIN'}
FIRST(STATEMENT_LIST) = {'EPSILON', 'FOR', 'STRING', 'FLOAT', 'DO', 'RETURN', 'IF', 'INT', 'BOOL', 'SWITCH', 'WHILE', 'DOUBLE', 'CHAR', 'ID'}
FIRST(STATEMENT) = {'FOR', 'STRING', 'FLOAT', 'DO', 'RETURN', 'IF', 'INT', 'BOOL', 'SWITCH', 'WHILE', 'DOUBLE', 'CHAR', 'ID'}
FIRST(DECLARATION) = {'INT', 'BOOL', 'STRING', 'FLOAT', 'DOUBLE', 'CHAR'}
FIRST(IF_STATEMENT) = {'IF'}
FIRST(WHILE_STATEMENT) = {'WHILE'}
FIRST(DO_WHILE_STATEMENT) = {'DO'}
FIRST(FOR_STATEMENT) = {'FOR'}
FIRST(SWITCH_STATEMENT) = {'SWITCH'}
FIRST(RETURN_STATEMENT) = {'RETURN'}
FIRST(STATEMENT_ID_START) = {'ID'}
FIRST(STATEMENT_ID_TAIL) = {'MULT', 'MOD', 'SEMI', 'ASSIGN', 'DIV', 'MINUS', 'PLUS'}
FIRST(TERM_PRIME) = {'DIV', 'EPSILON', 'MULT', 'MOD'}
FIRST(TYPE) = {'INT', 'FLOAT', 'DOUBLE', 'CHAR', 'BOOL', 'STRING'}
FIRST(DECLARATION_TAIL) = {'ASSIGN', 'SEMI'}
FIRST(ELSE_CLAUSE) = {'ELSE', 'EPSILON'}
FIRST(FOR_INIT) = {'INT', 'EPSILON', 'BOOL', 

In [ ]:
print("\n>>> CONJUNTO FOLLOW <<<")
for nao_terminal, conjunto in follow.items():
    # Só imprime os Não-Terminais
    if nao_terminal in grammar:
        print(f"FOLLOW({nao_terminal}) = {conjunto}")


>>> CONJUNTO FOLLOW <<<
FOLLOW(PROGRAM) = {'$'}
FOLLOW(MAIN_FUNCTION) = {'EOF'}
FOLLOW(STATEMENT_LIST) = {'BREAK', 'RCURLY'}
FOLLOW(STATEMENT) = {'BREAK', 'RCURLY', 'FOR', 'STRING', 'FLOAT', 'DO', 'RETURN', 'IF', 'INT', 'BOOL', 'SWITCH', 'WHILE', 'DOUBLE', 'CHAR', 'ID'}
FOLLOW(DECLARATION) = {'BREAK', 'RCURLY', 'FOR', 'STRING', 'FLOAT', 'DO', 'RETURN', 'IF', 'INT', 'BOOL', 'SWITCH', 'WHILE', 'SEMI', 'DOUBLE', 'CHAR', 'ID'}
FOLLOW(IF_STATEMENT) = {'BREAK', 'RCURLY', 'FOR', 'STRING', 'FLOAT', 'DO', 'RETURN', 'IF', 'INT', 'BOOL', 'SWITCH', 'WHILE', 'DOUBLE', 'CHAR', 'ID'}
FOLLOW(WHILE_STATEMENT) = {'BREAK', 'RCURLY', 'FOR', 'STRING', 'FLOAT', 'DO', 'RETURN', 'IF', 'INT', 'BOOL', 'SWITCH', 'WHILE', 'DOUBLE', 'CHAR', 'ID'}
FOLLOW(DO_WHILE_STATEMENT) = {'BREAK', 'RCURLY', 'FOR', 'STRING', 'FLOAT', 'DO', 'RETURN', 'IF', 'INT', 'BOOL', 'SWITCH', 'WHILE', 'DOUBLE', 'CHAR', 'ID'}
FOLLOW(FOR_STATEMENT) = {'BREAK', 'RCURLY', 'FOR', 'STRING', 'FLOAT', 'DO', 'RETURN', 'IF', 'INT', 'BOOL', 'SWITCH',

In [ ]:
print("\n=== 3. CONSTRUINDO TABELA ===")
build_parsing_table()

print("\n>>> TABELA DE PARSING (M) <<<")
sorted_keys = sorted(parsing_table.keys())
for key in sorted_keys:
    nao_terminal = key[0]
    terminal = key[1]
    prod = parsing_table[key]
    prod_str = " ".join(prod)
    print(f"M[{nao_terminal:<20}, {terminal:<10}] = {nao_terminal} -> {prod_str}")


=== 3. CONSTRUINDO TABELA ===

>>> TABELA DE PARSING (M) <<<
M[ASSIGNMENT          , ID        ] = ASSIGNMENT -> ID ASSIGN EXPRESSION
M[CASE_LIST           , CASE      ] = CASE_LIST -> CASE LITERAL COLON STATEMENT_LIST BREAK SEMI CASE_LIST
M[CASE_LIST           , DEFAULT   ] = CASE_LIST -> DEFAULT COLON STATEMENT_LIST BREAK SEMI
M[CASE_LIST           , RCURLY    ] = CASE_LIST -> EPSILON
M[CONDITION           , BOOL_LITERAL] = CONDITION -> LOGICAL_OR_EXPR
M[CONDITION           , CHAR_LITERAL] = CONDITION -> LOGICAL_OR_EXPR
M[CONDITION           , ID        ] = CONDITION -> LOGICAL_OR_EXPR
M[CONDITION           , LPAREN    ] = CONDITION -> LOGICAL_OR_EXPR
M[CONDITION           , MINUS     ] = CONDITION -> LOGICAL_OR_EXPR
M[CONDITION           , NOT       ] = CONDITION -> LOGICAL_OR_EXPR
M[CONDITION           , NUMBER    ] = CONDITION -> LOGICAL_OR_EXPR
M[CONDITION           , STRING_LITERAL] = CONDITION -> LOGICAL_OR_EXPR
M[DECLARATION         , BOOL      ] = DECLARATION -> TYPE ID DECL

In [ ]:
print("\n=== 4. PARSER (PILHA) ===")
parse(tokens)


=== 4. PARSER (PILHA) ===

PILHA                                                                  | ENTRADA              | AÇÃO
--------------------------------------------------------------------------------------------------------------
PROGRAM                                                                | main                 | PROGRAM -> MAIN_FUNCTION EOF
EOF MAIN_FUNCTION                                                      | main                 | MAIN_FUNCTION -> MAIN LPAREN RPAREN LCURLY STATEMENT_LIST RCURLY
EOF RCURLY STATEMENT_LIST LCURLY RPAREN LPAREN MAIN                    | main                 | Ler MAIN
EOF RCURLY STATEMENT_LIST LCURLY RPAREN LPAREN                         | (                    | Ler LPAREN
EOF RCURLY STATEMENT_LIST LCURLY RPAREN                                | )                    | Ler RPAREN
EOF RCURLY STATEMENT_LIST LCURLY                                       | {                    | Ler LCURLY
EOF RCURLY STATEMENT_LIST                       

False